In [2]:
import pandas as pd
from pathlib import Path
import os

def get_dataset_reference():
    current_folder = Path(os.getcwd())
    model_folder = current_folder.parent
    webscrapping_folder = model_folder.parent
    return Path(webscrapping_folder, "matches", "datasets")

In [3]:
filename = "5000.csv"
df = pd.read_csv(f"{get_dataset_reference()}\\{filename}")
df

,RegularTime,SpikeTime,MapName,FinalWinner,RoundID,MatchID,RoundNumber,RoundTime,ATK_loadoutValue,ATK_weaponValue,...,DEF_remainingCreds,DEF_operators,DEF_Initiator,DEF_Duelist,DEF_Sentinel,DEF_Controller,Team_A_ID,Team_A_Name,Team_B_ID,Team_B_Name
0,0,0,Split,1,107567,6197,1,0,3900,0,...,0,0,1,2,1,1,279,Luminosity Gaming,32,Equinox Esports
1,15,0,Split,1,107567,6197,1,15494,3900,0,...,0,0,1,2,1,0,279,Luminosity Gaming,32,Equinox Esports
2,16,0,Split,1,107567,6197,1,16500,3900,0,...,0,0,1,1,1,0,279,Luminosity Gaming,32,Equinox Esports
3,26,0,Split,1,107567,6197,1,25999,3900,0,...,0,0,1,1,0,0,279,Luminosity Gaming,32,Equinox Esports
4,32,0,Split,1,107567,6197,1,32331,3900,0,...,0,0,0,1,0,0,279,Luminosity Gaming,32,Equinox Esports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898184,0,7,Icebox,0,494951,31800,14,21175,8150,5300,...,100,0,1,1,1,0,4307,Project ABC,4308,Havik Esports
898185,0,8,Icebox,0,494951,31800,14,22281,4600,3200,...,100,0,1,1,1,0,4307,Project ABC,4308,Havik Esports
898186,0,9,Icebox,0,494951,31800,14,23697,2400,1600,...,100,0,1,1,1,0,4307,Project ABC,4308,Havik Esports
898187,0,18,Icebox,0,494951,31800,14,31960,0,0,...,100,0,1,1,1,0,4307,Project ABC,4308,Havik Esports


# <font color='green'>Feature selection</font>